# Support Vector Classifier using La Classy

In [1]:
import { parse } from "https://deno.land/std@0.204.0/csv/parse.ts";
import {
  ClassificationReport,
  TextVectorizer,
  Matrix,
  useSplit,
} from "https://deno.land/x/vectorizer@v0.3.4/mod.ts";
import {
  GradientDescentSolver,
  hinge,
} from "https://deno.land/x/classylala@v1.0.0/mod.ts";


We first load our dataset `spam.csv`.

In [2]:
const data = parse(Deno.readTextFileSync("../datasets/spam.csv"));

Skip the first row (header).

In [3]:
data.shift()

[ "label", "content" ]

We can now get the predictor and target variables from the dataset.

In [4]:
const x = data.map((fl, i) => fl[1]);
x.slice(0, 10)

[
  "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got a"... 11 more characters,
  "Ok lar... Joking wif u oni...",
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entr"... 55 more characters,
  "U dun say so early hor... U c already then say...",
  "Nah I don't think he goes to usf, he lives around here though",
  "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it s"... 47 more characters,
  "Even my brother is not like to speak with me. They treat me like aids patent.",
  "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertu"... 60 more characters,
  "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim"... 57 more characters,
  "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera f"... 54 more ch

In [5]:
const y = new Matrix(Float64Array.from(data.map((fl) => fl[0] === "spam" ? 1 : -1)), [data.length]);
y.slice(0, 10)

idx,0
0,-1
1,-1
2,1
3,-1
4,-1
5,1
6,-1
7,-1
8,1
9,1


In [6]:
y.shape

[ 5572, 1 ]

We now split our dataset for training and testing purposes. 

In [7]:
const [[x_train, y_train], [x_test, y_test]] = useSplit(
  { ratio: [7, 3], shuffle: true },
  x,
  y
);
x_train.slice(0, 10)

[
  "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got a"... 11 more characters,
  "Ok lar... Joking wif u oni...",
  "U dun say so early hor... U c already then say...",
  "Nah I don't think he goes to usf, he lives around here though",
  "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it s"... 47 more characters,
  "Even my brother is not like to speak with me. They treat me like aids patent.",
  "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim"... 57 more characters,
  "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera f"... 54 more characters,
  "SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6day"... 36 more characters,
  "I've been searching for the right words to thank you for this breather. I promise i wont take your h"... 96 more ch

Our input variables are fully text. In order to convert them to vectors, we use a `TextVectorizer`.

In [8]:
import { TextVectorizer } from "https://deno.land/x/vectorizer@v0.3.4/mod.ts";

const vec = new TextVectorizer({
  mode: "tfidf",
  config: { skipWords: "english", standardize: { lowercase: true } },
});
const X_train = vec.fit(x_train).transform<"f64">(x_train, "f64");
const X_test = vec.transform<"f64">(x_test, "f64")
X_train.slice(0, 10);

4.075774981227528	9.268731832117737	9.268731832117737	9.268731832117737	6.783825182329737	8.170119543449628	4.624340932976365	5.1255971057262055	6.435518488061522	8.575584651557794	5.3975308212098465	9.268731832117737	9.268731832117737	4.193558016883911	9.268731832117737	7.322821683062425	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	

Now that we have prepared our inputs, we can initialize our solver.

We use the `hinge` loss function which is used for binary classification with SVM.

In [9]:
const solver = new GradientDescentSolver({
  loss: hinge(),
});


We can then train our model using the data we acquired.

Setting the learning rate to a small value is desirable. We are training our model for 100 epochs with 20 minibatches.

In [10]:
solver.train(X_train, y_train, {
  learning_rate: 0.01,
  epochs: 100,
  n_batches: 20,
  patience: 10
});

The model is trained, now it is time to evaluate its performance on our testing dataset

In [11]:
const res = solver.predict(X_test)
res.shape

[ 1672, 1 ]

In [12]:
res.row(0)

Float64Array(1) [ 112.95470391868602 ]

Minimizing the `hinge` loss increases the margin between positive and negative samples. We take any positive output as a positive example and negative output as a negative one.

In [13]:
for (let i = 0; i < res.data.length; i += 1) {
  res.data[i] = res.data[i] > 0 ? 1 : -1
}
res.slice(0, 10)

idx,0
0,1
1,-1
2,-1
3,1
4,1
5,-1
6,-1
7,-1
8,-1
9,-1


Finally, we can generate a classification report based on our results.

In [14]:
new ClassificationReport(y.data, res.data)

Classification Report
Number of classes:	2
Class	Preci	F1	Rec	Sup
-1	0.8598679383712399	0.37879767291531996	0.24290155440414507	4825
1	0.7396039603960396	0.8503130335799659	1	747
Accuracy		0.6314608758076095	5572